In [4]:
from cluster.RayJobManager import RayJobManager

In [5]:
RAY_ADDRESS  = "http://10.100.192.238:8265"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "torch",
        "ray[tune]",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--verbose"]
}

ray_manager = RayJobManager(RAY_ADDRESS)

In [6]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

ray_manager.wait_until_done(job_id)

2025-11-03 16:25:30,306	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_ef417cab60578c05.zip.
2025-11-03 16:25:30,308	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_nQ9n416iaix1QSsH
Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING


KeyboardInterrupt: 